# Optimization and Simulation with DR

## optunaによるアプローチ

### 1. ライブラリ類のインポートとファイルのロード

In [1]:
%matplotlib inline

import os
import copy
import warnings

import numpy as np
import pandas as pd
from pandas import DataFrame

import matplotlib.pyplot as plt
import datarobot as dr
from datarobot_predict.deployment import predict

import optuna

import japanize_matplotlib

if not os.getenv("DATAROBOT_NOTEBOOK_IMAGE"):
    print("not running in DataRobot Notebook")
    from dotenv import load_dotenv
    load_dotenv("../.env", override=True)

client = dr.Client()

seed = 71
np.random.seed(seed)
warnings.filterwarnings("ignore")
plt.rcParams.update({"figure.max_open_warning": 0})
optuna.visualization.is_available()
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)
pd.set_option("display.precision", 8)

not running in DataRobot Notebook


In [2]:
targets = ["降伏強度", "引張強度"]

df = pd.read_csv("../data/opt_steel_strength.csv")

In [3]:
X_train = df.drop(["ID", "降伏強度", "引張強度"], axis=1)

### 2. 予測APIを用いる上での必要な情報を準備しておきます。

In [4]:
# 予測APIを叩くための諸々を設定する必要があります
dr.Client()
deployment_ids = ["67bc6ab2f1b3f6b73d560ff8", "67bc6ab999dbbff523b1152a"]
deployment_quant_ids = ["67bc6ad2fd8c6942c3561092", "67bc6ad599dbbff523b11589"]
deployments = [dr.Deployment.get(deployment_id) for deployment_id in deployment_ids]
deployments_quant = [
    dr.Deployment.get(deployment_id) for deployment_id in deployment_quant_ids
]
# 実験時間、実験回数（n_trials）での制御も可能
timeout = 120

# 結果格納用
result = pd.DataFrame()

### 3. `datarobot-predict`を使って予測APIを叩く

In [5]:
# 一回叩いてみる
predictions, _ = predict(deployments[1], X_train.iloc[:1])
predictions

,引張強度_PREDICTION,DEPLOYMENT_APPROVAL_STATUS
0,2442.85464807,APPROVED


In [6]:
# 予測の戻り値から予測値を取り出す
predictions.loc[0, "引張強度_PREDICTION"]

np.float64(2442.854648066)

### タスク1
`引張強度`に対して最適化関数を定義

#### Pruner使わない場合の最適化関数

In [ ]:
# 最適化の対象。この関数の戻り値を最大/小化します。
# Optunaの場合、trailにいろんな情報格納くれています。

stat = X_train.describe().T


def objective(trial):

    df_target = DataFrame(index=[0], columns=X_train.columns)
    for col in X_train.columns:
        low = stat.loc[col, "min"] * 0.8
        high = stat.loc[col, "max"] * 1.2
        df_target[col] = trial.suggest_float(col, low, high, step=0.01)

    pred_2 = predict(deployments[1], df_target)[0].loc[
        0, "XXXX ここの記入してください XXXX"
    ]

    return pred_2

#### Pruner使う場合の最適化関数

In [ ]:
# 打ち止めするかを確認するため、途中経過をフィードバックするようにobjectiveを編集

n_train_iter = (
    "XXXX ここの記入してください XXXX"  # 文字列ではなく数値として入れてください
)


def objective_prune(trial):

    df_target = DataFrame(index=[0], columns=X_train.columns)

    for col in X_train.columns:
        low = stat.loc[col, "min"] * 0.8
        high = stat.loc[col, "max"] * 1.2
        df_target[col] = trial.suggest_float(col, low, high, step=0.01)

    for step in range(n_train_iter):
        pred_2 = predict(deployments[1], df_target)[0].loc[
            0, "XXXX ここの記入してください XXXX"
        ]

        # 打ち止めかどうかを判断
        pred_temp = pred_2

        trial.report(pred_temp, step)

        if trial.should_prune():
            raise optuna.TrialPruned()

    return pred_2

### タスク2
`引張強度`に対して最適化を実施する

下記4のSamplerを使って最適化し、結果を記録する。
- [RandomSampler](https://optuna.readthedocs.io/en/v2.10.1/reference/generated/optuna.samplers.RandomSampler.html#optuna.samplers.RandomSampler)
- [TPESampler](https://optuna.readthedocs.io/en/v2.10.1/reference/generated/optuna.samplers.TPESampler.html#optuna.samplers.TPESampler)
- [CmaEsSampler](https://optuna.readthedocs.io/en/v2.10.1/reference/generated/optuna.samplers.CmaEsSampler.html#optuna.samplers.CmaEsSampler)<br>
- [AutoSampler](https://hub.optuna.org/samplers/auto_sampler/)

要件
- Samplerのパラメーターはドキュメント参照して変更OK、初期設定をそのまま使うのも大丈夫。
- シードは必ず`seed`（=71）に固定する
- `study.optimize`で最適化する際、`n_jobs`を3まで調整OK（ローカルで実行する場合は特に制限しません）
- **4**通り以上`result`に記録してください

In [ ]:
%%time
# パターン1
name = "XXXX ここにsamplerの名前を記入してください XXXX"
n_jobs = "XXXX ここに並列数を記入してください XXXX"
sampler = optuna.samplers."XXXX ここにクラス名を記入してください XXXX"(seed=seed)

study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, timeout=timeout, gc_after_trial=True, n_jobs=n_jobs)
_result = study.trials_dataframe()
_result["name"] = name
result = pd.concat([result, _result], axis=0, ignore_index=True)

In [ ]:
%%time
# パターン2
name = "XXXX ここにsamplerの名前を記入してください XXXX"
n_jobs = "XXXX ここに並列数を記入してください XXXX"
sampler = optuna.samplers."XXXX ここにクラス名を記入してください XXXX"(seed=seed)

study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, timeout=timeout, gc_after_trial=True, n_jobs=n_jobs)
_result = study.trials_dataframe()
_result["name"] = name
result = pd.concat([result, _result], axis=0, ignore_index=True)

In [ ]:
%%time
# パターン3
name = "XXXX ここにsamplerの名前を記入してください XXXX"
n_jobs = "XXXX ここに並列数を記入してください XXXX"
sampler = optuna.samplers."XXXX ここにクラス名を記入してください XXXX"(seed=seed)

study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, timeout=timeout, gc_after_trial=True, n_jobs=n_jobs)
_result = study.trials_dataframe()
_result["name"] = name
result = pd.concat([result, _result], axis=0, ignore_index=True)

In [ ]:
%%time
# パターン4
# AutoSamplerはoptunahubからロードできます。
name = "XXXX ここにsamplerの名前を記入してください XXXX"
n_jobs = "XXXX ここに並列数を記入してください XXXX"

module = optunahub.load_module(package="XXXX ここにクラス名を記入してください XXXX")
study = optuna.create_study(sampler=module."XXXX ここにクラス名を記入してください XXXX"(seed=seed),direction="maximize")
study.optimize(objective, timeout=timeout, gc_after_trial=True, n_jobs=n_jobs)
_result = study.trials_dataframe()
_result["name"] = name
result = pd.concat([result, _result], axis=0, ignore_index=True)

### タスク3
`引張強度`に対して枝刈りを使って最適化を実施する

タスク２でリストアップしたSamplerと下記のPrunerと組み合わせして最適化し、結果を記録する
- [MedianPruner](https://optuna.readthedocs.io/en/v2.10.1/reference/generated/optuna.pruners.MedianPruner.html#optuna.pruners.MedianPruner)
- [PatientPruner](https://optuna.readthedocs.io/en/v2.10.1/reference/generated/optuna.pruners.PatientPruner.html#optuna.pruners.PatientPruner)
- [PercentilePruner](https://optuna.readthedocs.io/en/v2.10.1/reference/generated/optuna.pruners.PercentilePruner.html#optuna.pruners.PercentilePruner)
- [SuccessiveHalvingPruner](https://optuna.readthedocs.io/en/v2.10.1/reference/generated/optuna.pruners.SuccessiveHalvingPruner.html#optuna.pruners.SuccessiveHalvingPruner)
- [HyperbandPruner](https://optuna.readthedocs.io/en/v2.10.1/reference/generated/optuna.pruners.HyperbandPruner.html#optuna.pruners.HyperbandPruner)

要件
- SamplerとPrunerのパラメーターはドキュメント参照して変更OK、初期設定をそのまま使うのも大丈夫
- シードは必ず`seed`（=71）に固定する
- `study.optimize`で最適化する際、`n_jobs`を3まで調整OK
- **2**通り以上`result`に記録してください

In [ ]:
%%time
# パターン1
name_sampler = "XXXX ここにsamplerの名前を記入してください XXXX"
name_pruner = "XXXX ここにprunerの名前を記入してください XXXX"
n_jobs = "XXXX ここに並列数を記入してください XXXX"
sampler = optuna.samplers."XXXX ここにクラス名を記入してください XXXX"(seed=seed)
pruner = optuna.pruners."XXXX ここにクラス名を記入してください XXXX"()
study = optuna.create_study(direction="maximize", sampler=sampler, pruner=pruner)
study.optimize(objective, timeout=timeout, gc_after_trial=True, n_jobs=n_jobs)
_result = study.trials_dataframe()
_result["name"] = name_sampler+"_"+name_pruner
result = pd.concat([result, _result], axis=0, ignore_index=True)

In [ ]:
%%time
# パターン2
name_sampler = "XXXX ここにsamplerの名前を記入してください XXXX"
name_pruner = "XXXX ここにprunerの名前を記入してください XXXX"
n_jobs = "XXXX ここに並列数を記入してください XXXX"
sampler = optuna.samplers."XXXX ここにクラス名を記入してください XXXX"(seed=seed)
pruner = optuna.pruners."XXXX ここにクラス名を記入してください XXXX"()
study = optuna.create_study(direction="maximize", sampler=sampler, pruner=pruner)
study.optimize(objective, timeout=timeout, gc_after_trial=True, n_jobs=n_jobs)
result = update_result(result, study, name_sampler, name_pruner, n_jobs)
_result = study.trials_dataframe()
_result["name"] = name_sampler+"_"+name_pruner
result = pd.concat([result, _result], axis=0, ignore_index=True)

### 結果表示

In [ ]:
df_result = pd.DataFrame(result)
display(df_result)

In [ ]:
# 一番良かった結果を見てみよう
df_result.loc[df_result["value"] == df_result["value"].max()]

In [ ]:
# This is the end of this code...